In [4]:
import cv2
import numpy as np
import os

def get_mask_of_board(frame_bgr):
    """Extract board mask using HSV color filtering"""
    hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
    
    # Create mask for board colors (green/yellow/blue - typical board colors)
    mask = cv2.inRange(hsv, np.array([20, 40, 40]), np.array([140, 255, 255]))
    
    # Morphological operations to clean up the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter by area
    filtered = [c for c in contours if cv2.contourArea(c) > 500]
    
    if not filtered:
        return None
    
    # Combine all contours and create convex hull
    all_points = np.vstack(filtered)
    hull = cv2.convexHull(all_points)
    
    # Create final board mask
    board_mask = np.zeros_like(mask)
    cv2.drawContours(board_mask, [hull], -1, 255, -1)
    
    return board_mask


def detect_board_simple_mask(frame):
    """Simple board detection using just the color mask"""
    
    board_mask = get_mask_of_board(frame)
    
    if board_mask is None:
        return None
    
    # Find the convex hull contour
    contours, _ = cv2.findContours(board_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    # Get largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Approximate to quadrilateral
    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)
    
    # If we get 4 corners, use those; otherwise use bounding rect
    if len(approx) == 4:
        return approx.reshape(-1, 1, 2).astype(np.float32)
    else:
        # Use minimum area rectangle
        rect = cv2.minAreaRect(largest_contour)
        box = cv2.boxPoints(rect)
        return box.reshape(-1, 1, 2).astype(np.float32)


def detect_cup(frame, cup_template):
    """Detect the dice cup as a full black circle"""
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect black/very dark colors more strictly
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 60])
    mask = cv2.inRange(hsv, lower_black, upper_black)
    
    # Use gentler morphological operations to preserve circular shape
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None, 0
    
    # Look for circular contours only
    cup_candidates = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if 800 < area < 25000:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h if h > 0 else 0
            
            # Calculate circularity
            perimeter = cv2.arcLength(contour, True)
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter ** 2)
            else:
                circularity = 0
            
            # STRICT circular requirements only
            is_circular = (0.75 < aspect_ratio < 1.25) and (circularity > 0.75)
            
            if is_circular:
                # Calculate confidence heavily weighted on circularity
                confidence = circularity * 0.85 + (1.0 - abs(1.0 - aspect_ratio)) * 0.15
                
                # Additional check: filled black circle
                contour_mask = np.zeros(gray.shape, dtype=np.uint8)
                cv2.drawContours(contour_mask, [contour], -1, 255, -1)
                
                # Check darkness
                roi = cv2.bitwise_and(gray, gray, mask=contour_mask)
                mean_brightness = cv2.mean(roi, mask=contour_mask)[0]
                
                if mean_brightness < 80:
                    darkness_score = 1.0 - (mean_brightness / 255.0)
                    confidence = confidence * 0.7 + darkness_score * 0.3
                    cup_candidates.append(((x, y, w, h), confidence, area))
    
    if not cup_candidates:
        return None, 0
    
    # Return the best candidate
    cup_candidates.sort(key=lambda x: x[1], reverse=True)
    best_cup = cup_candidates[0]
    
    return best_cup[0], best_cup[1]


def detect_dice_by_dots(frame, search_region=None, cup_size=None):
    """Detect dice by finding the black square and counting white dots inside"""
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If search region provided, crop to that area
    if search_region is not None:
        x_offset, y_offset, w_region, h_region = search_region
        x_offset = max(0, x_offset)
        y_offset = max(0, y_offset)
        w_region = min(w_region, gray.shape[1] - x_offset)
        h_region = min(h_region, gray.shape[0] - y_offset)
        gray_search = gray[y_offset:y_offset+h_region, x_offset:x_offset+w_region]
        hsv_search = hsv[y_offset:y_offset+h_region, x_offset:x_offset+w_region]
    else:
        gray_search = gray
        hsv_search = hsv
        x_offset, y_offset = 0, 0
    
    # Step 1: Find very dark/black regions (the dice body)
    # Use grayscale for more reliable black detection
    _, black_mask = cv2.threshold(gray_search, 70, 255, cv2.THRESH_BINARY_INV)
    
    # Also try HSV-based detection for very dark colors
    lower_dark = np.array([0, 0, 0])
    upper_dark = np.array([180, 255, 70])
    hsv_black_mask = cv2.inRange(hsv_search, lower_dark, upper_dark)
    
    # Combine both masks
    dark_mask = cv2.bitwise_or(black_mask, hsv_black_mask)
    
    # Clean up the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dark_mask = cv2.morphologyEx(dark_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    dark_mask = cv2.morphologyEx(dark_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # Find contours of dark regions
    contours, _ = cv2.findContours(dark_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    dice_candidates = []
    
    # Calculate max dice size based on cup size (dice is ALWAYS smaller than cup)
    min_dice_area = 500  # Much smaller minimum
    max_dice_area = 8000  # Default
    if cup_size is not None:
        cup_w, cup_h = cup_size
        cup_area = cup_w * cup_h
        max_dice_area = cup_area * 0.5  # Dice is at most 50% of cup area
        min_dice_area = cup_area * 0.05  # At least 5% of cup
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        # Dice should be smaller than cup and reasonably sized
        if min_dice_area < area < max_dice_area:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h if h > 0 else 0
            
            # Additional check: if cup exists, dice must be significantly smaller
            if cup_size is not None:
                cup_w, cup_h = cup_size
                if w >= cup_w * 0.9 or h >= cup_h * 0.9:
                    continue  # Skip if too close to cup size
            
            # Dice should be roughly square (more lenient)
            if 0.6 < aspect_ratio < 1.6:
                # Extract the region
                dice_roi = gray_search[y:y+h, x:x+w]
                hsv_roi = hsv_search[y:y+h, x:x+w]
                
                # Step 2: Find white/light dots inside this region
                # Use both grayscale and HSV for better detection
                _, white_mask_gray = cv2.threshold(dice_roi, 160, 255, cv2.THRESH_BINARY)
                
                lower_white = np.array([0, 0, 160])
                upper_white = np.array([180, 60, 255])
                white_mask_hsv = cv2.inRange(hsv_roi, lower_white, upper_white)
                
                # Combine both
                white_mask = cv2.bitwise_or(white_mask_gray, white_mask_hsv)
                
                # Clean up dot mask
                kernel_small = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
                white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel_small)
                white_mask = cv2.morphologyEx(white_mask, cv2.MORPH_CLOSE, kernel_small)
                
                # Find dot contours
                dot_contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                # Count circular dots
                valid_dots = []
                for dot_contour in dot_contours:
                    dot_area = cv2.contourArea(dot_contour)
                    
                    # More lenient dot size requirements
                    if 10 < dot_area < 1000:
                        dot_x, dot_y, dot_w, dot_h = cv2.boundingRect(dot_contour)
                        dot_aspect = float(dot_w) / dot_h if dot_h > 0 else 0
                        
                        # More lenient circularity check
                        perimeter = cv2.arcLength(dot_contour, True)
                        if perimeter > 0:
                            circularity = 4 * np.pi * dot_area / (perimeter ** 2)
                        else:
                            circularity = 0
                        
                        if 0.4 < dot_aspect < 2.0 and circularity > 0.3:
                            valid_dots.append(dot_contour)
                
                num_dots = len(valid_dots)
                
                # Valid dice have 1-6 dots
                if 1 <= num_dots <= 6:
                    # Calculate confidence
                    square_score = 1.0 - abs(1.0 - aspect_ratio)
                    confidence = square_score * 0.5 + (num_dots / 6.0) * 0.5
                    
                    # Adjust position to global coordinates
                    global_x = x + x_offset
                    global_y = y + y_offset
                    
                    dice_candidates.append({
                        'number': num_dots,
                        'location': (global_x, global_y, w, h),
                        'confidence': confidence,
                        'dots': valid_dots,
                        'roi_offset': (x, y),
                        'area': area
                    })
                # If we find a dark square but no dots, still consider it with lower confidence
                elif num_dots == 0 and area > 200:
                    # Might be a dice with dots we can't see clearly
                    # Check if it's very dark
                    mean_brightness = cv2.mean(dice_roi)[0]
                    if mean_brightness < 50:
                        global_x = x + x_offset
                        global_y = y + y_offset
                        dice_candidates.append({
                            'number': 1,  # Assume 1 dot as fallback
                            'location': (global_x, global_y, w, h),
                            'confidence': 0.3,
                            'dots': [],
                            'roi_offset': (x, y),
                            'area': area
                        })
    
    if not dice_candidates:
        return None, 0, None
    
    # Prioritize by confidence, but also prefer smaller objects (dice is small!)
    # If cup size is known, prefer candidates that are much smaller than cup
    if cup_size is not None:
        cup_area = cup_size[0] * cup_size[1]
        for candidate in dice_candidates:
            size_ratio = candidate['area'] / cup_area
            # Boost confidence for appropriately sized objects
            if 0.1 < size_ratio < 0.4:
                candidate['confidence'] *= 1.3
    
    # Return the best candidate (highest confidence)
    dice_candidates.sort(key=lambda d: d['confidence'], reverse=True)
    best = dice_candidates[0]
    
    return best['number'], best['confidence'], best['location']


def draw_detections(frame, board_corners, cup_info, dice_info, show_mask=False):
    """Draw the detected board, cup, and dice on the frame"""
    
    output = frame.copy()
    
    if show_mask:
        mask = get_mask_of_board(frame)
        if mask is not None:
            mask_colored = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
            mask_colored[:, :, 0] = 0
            mask_colored[:, :, 1] = mask
            output = cv2.addWeighted(output, 0.7, mask_colored, 0.3, 0)
    
    # Draw board
    if board_corners is not None:
        pts = board_corners.reshape(-1, 2).astype(np.int32)
        cv2.polylines(output, [pts], True, (0, 255, 0), 3)
        
        for i, pt in enumerate(pts):
            cv2.circle(output, tuple(pt), 8, (0, 0, 255), -1)
            cv2.putText(output, str(i+1), tuple(pt + 15), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Draw cup
    if cup_info[0] is not None:
        x, y, w, h = cup_info[0]
        confidence = cup_info[1]
        
        cv2.rectangle(output, (x, y), (x+w, y+h), (255, 0, 255), 3)
        
        cx, cy = x + w//2, y + h//2
        cv2.circle(output, (cx, cy), 5, (255, 0, 255), -1)
        
        label = f"Cup ({confidence:.2f})"
        cv2.putText(output, label, (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
    
    # Draw dice
    if dice_info[0] is not None:
        dice_num, dice_score, dice_loc = dice_info
        if dice_loc is not None:
            x, y, w, h = dice_loc
            cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 255), 4)
            
            # Draw a filled background for better visibility
            label = f"DICE: {dice_num}"
            (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)
            cv2.rectangle(output, (x, y-label_h-20), (x+label_w+10, y-5), (0, 0, 0), -1)
            cv2.putText(output, label, (x+5, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)
    
    return output


def process_video_realtime(video_path, board_template_path, cup_template_path):
    """Process video and show real-time detection"""
    
    # Load board template
    board_template = cv2.imread(board_template_path)
    if board_template is None:
        print(f"Error: Could not load board template from {board_template_path}")
        return
    print(f"Board template loaded: {board_template.shape}")
    
    # Load cup template
    cup_template = cv2.imread(cup_template_path)
    if cup_template is None:
        print(f"Error: Could not load cup template from {cup_template_path}")
        return
    print(f"Cup template loaded: {cup_template.shape}")
    
    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {width}x{height} @ {fps} FPS, Total frames: {total_frames}")
    print("\nControls:")
    print("  SPACE - Pause/Resume")
    print("  Q - Quit")
    print("  R - Reset to beginning")
    print("  M - Toggle mask overlay")
    
    frame_num = 0
    board_detection_count = 0
    cup_detection_count = 0
    dice_detection_count = 0
    paused = False
    show_mask = False
    
    print("\nProcessing video...")
    print("Dice detection uses dot counting (no templates needed)")
    
    while True:
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("\nEnd of video reached. Press 'R' to restart or 'Q' to quit.")
                paused = True
                continue
        
        # Detect board
        board_corners = detect_board_simple_mask(frame)
        if board_corners is not None:
            board_detection_count += 1
        
        # Detect cup
        cup_rect, cup_confidence = detect_cup(frame, cup_template)
        if cup_rect is not None:
            cup_detection_count += 1
        
        # Detect dice (search in expanded region around cup, or whole frame)
        search_region = None
        cup_size = None
        if cup_rect is not None:
            x, y, w, h = cup_rect
            cup_size = (w, h)
            margin = 200  # Larger search area
            search_region = (max(0, x-margin), max(0, y-margin), 
                           w + 2*margin, h + 2*margin)
        
        dice_num, dice_score, dice_loc = detect_dice_by_dots(frame, search_region, cup_size)
        if dice_num is not None:
            dice_detection_count += 1
        
        # Draw detections
        output_frame = draw_detections(frame, board_corners, 
                                      (cup_rect, cup_confidence), 
                                      (dice_num, dice_score, dice_loc),
                                      show_mask)
        
        # Add frame info
        info_text = f"Frame: {frame_num}/{total_frames}"
        cv2.putText(output_frame, info_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        board_text = f"Board: {'DETECTED' if board_corners is not None else 'NOT FOUND'}"
        board_color = (0, 255, 0) if board_corners is not None else (0, 0, 255)
        cv2.putText(output_frame, board_text, (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, board_color, 2)
        
        cup_text = f"Cup: {'DETECTED' if cup_rect is not None else 'NOT FOUND'}"
        cup_color = (255, 0, 255) if cup_rect is not None else (0, 0, 255)
        cv2.putText(output_frame, cup_text, (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, cup_color, 2)
        
        dice_text = f"Dice: {dice_num if dice_num is not None else 'NOT FOUND'}"
        dice_color = (0, 255, 255) if dice_num is not None else (0, 0, 255)
        cv2.putText(output_frame, dice_text, (10, 120),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, dice_color, 2)
        
        # Add pause indicator
        if paused:
            cv2.putText(output_frame, "PAUSED", (width - 150, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        # Show frame
        cv2.imshow('Ludo Board Detection', output_frame)
        
        # Handle keyboard input
        key = cv2.waitKey(30 if not paused else 0) & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord(' '):
            paused = not paused
        elif key == ord('r'):
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            frame_num = 0
            board_detection_count = 0
            cup_detection_count = 0
            dice_detection_count = 0
            paused = False
        elif key == ord('m'):
            show_mask = not show_mask
            print(f"Mask overlay: {'ON' if show_mask else 'OFF'}")
        
        if not paused:
            frame_num += 1
            
            if frame_num % 30 == 0:
                progress = (frame_num / total_frames) * 100
                board_rate = (board_detection_count / frame_num) * 100 if frame_num > 0 else 0
                cup_rate = (cup_detection_count / frame_num) * 100 if frame_num > 0 else 0
                dice_rate = (dice_detection_count / frame_num) * 100 if frame_num > 0 else 0
                print(f"Progress: {progress:.1f}% | Board: {board_rate:.1f}% | Cup: {cup_rate:.1f}% | Dice: {dice_rate:.1f}%")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    # Print summary
    board_rate = (board_detection_count / frame_num) * 100 if frame_num > 0 else 0
    cup_rate = (cup_detection_count / frame_num) * 100 if frame_num > 0 else 0
    dice_rate = (dice_detection_count / frame_num) * 100 if frame_num > 0 else 0
    print(f"\n{'='*50}")
    print(f"Processing stopped!")
    print(f"Frames processed: {frame_num}/{total_frames}")
    print(f"Board detections: {board_detection_count} ({board_rate:.1f}%)")
    print(f"Cup detections: {cup_detection_count} ({cup_rate:.1f}%)")
    print(f"Dice detections: {dice_detection_count} ({dice_rate:.1f}%)")
    print(f"{'='*50}")


def run_detection(video_path="pro1.mp4", board_path="board.jpg", cup_path="cup.jpg"):
    """Main function to call from Jupyter - No dice templates needed!"""
    
    # Check if files exist
    if not os.path.exists(board_path):
        print(f"Error: Board template not found at {board_path}")
        return
    
    if not os.path.exists(cup_path):
        print(f"Error: Cup template not found at {cup_path}")
        return
    
    if not os.path.exists(video_path):
        print(f"Error: Video not found at {video_path}")
        return
    
    print(f"Board template: {board_path}")
    print(f"Cup template: {cup_path}")
    print(f"Input video: {video_path}")
    print("Dice detection: Using dot counting (no templates needed)\n")
    
    process_video_realtime(video_path, board_path, cup_path)


# For running as script
if __name__ == "__main__":
    import sys
    
    # Filter out Jupyter kernel arguments
    args = [arg for arg in sys.argv[1:] if not arg.startswith('--') and not arg.startswith('-f=')]
    
    # Default paths
    board_path = "data/board.jpg"
    cup_path = "data/cup.jpg"
    video_path = "data/pro3.mp4"
    
    # Parse filtered arguments
    if len(args) > 0:
        video_path = args[0]
    if len(args) > 1:
        board_path = args[1]
    if len(args) > 2:
        cup_path = args[2]
    
    # Check if running in Jupyter
    if len(args) == 0 and len([arg for arg in sys.argv[1:] if arg.startswith('-f=')]) > 0:
        print("Running in Jupyter Notebook mode.")
        print("Use: run_detection('pro1.mp4', 'board.jpg', 'cup.jpg')")
    else:
        run_detection(video_path, board_path, cup_path)

Board template: data/board.jpg
Cup template: data/cup.jpg
Input video: data/pro3.mp4
Dice detection: Using dot counting (no templates needed)

Board template loaded: (3692, 3852, 3)
Cup template loaded: (939, 438, 3)
Video properties: 888x1920 @ 30 FPS, Total frames: 2089

Controls:
  SPACE - Pause/Resume
  Q - Quit
  R - Reset to beginning
  M - Toggle mask overlay

Processing video...
Dice detection uses dot counting (no templates needed)
Progress: 1.4% | Board: 100.0% | Cup: 100.0% | Dice: 70.0%
Progress: 2.9% | Board: 100.0% | Cup: 100.0% | Dice: 83.3%
Progress: 4.3% | Board: 100.0% | Cup: 100.0% | Dice: 88.9%
Progress: 5.7% | Board: 100.0% | Cup: 100.0% | Dice: 91.7%
Progress: 7.2% | Board: 100.0% | Cup: 100.0% | Dice: 93.3%

Processing stopped!
Frames processed: 164/2089
Board detections: 165 (100.6%)
Cup detections: 165 (100.6%)
Dice detections: 155 (94.5%)
